In [200]:
import pandas as pd
import sqlite3
import warnings

warnings.filterwarnings('ignore')

In [201]:
order = pd.read_excel('/content/drive/MyDrive/Тестовые/Тестовое задание (Ведущий аналитик)_.xlsx', sheet_name=2)
product = pd.read_excel('/content/drive/MyDrive/Тестовые/Тестовое задание (Ведущий аналитик)_.xlsx', sheet_name=3)

Посмотрим на данные и проведем EDA

In [202]:
order.head()

,order_id,accepted_at,product_id,quantity,regular_price,price,cost_price
0,1516729899,2022-01-10 16:37:37,10163,1,329.0,329.0,254
1,1518335746,2022-01-16 12:22:02,10163,1,329.0,329.0,229
2,1518354200,2022-01-15 17:03:41,10163,1,329.0,329.0,229
3,1516838503,2022-01-11 12:31:53,10163,1,329.0,329.0,254
4,1516739758,2022-01-10 19:33:35,10163,1,329.0,329.0,254


In [203]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23582 entries, 0 to 23581
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       23582 non-null  int64         
 1   accepted_at    23582 non-null  datetime64[ns]
 2   product_id     23582 non-null  int64         
 3   quantity       23582 non-null  int64         
 4   regular_price  23582 non-null  float64       
 5   price          23582 non-null  float64       
 6   cost_price     23582 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 1.3 MB


In [204]:
order.isna().sum()

order_id         0
accepted_at      0
product_id       0
quantity         0
regular_price    0
price            0
cost_price       0
dtype: int64

In [205]:
order.dtypes

order_id                  int64
accepted_at      datetime64[ns]
product_id                int64
quantity                  int64
regular_price           float64
price                   float64
cost_price                int64
dtype: object

In [206]:
product.head()

,product_id,level1,name
0,59272,Бакалея,Соус Ben's Original
1,9096,Бакалея,Киноа Агро-Альянс
2,2506,Бакалея,Рис Агро-Альянс
3,15017,Бакалея,Укроп Kotanyi
4,823,Бакалея,Макароны Шебекинские


In [207]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3573 entries, 0 to 3572
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  3573 non-null   int64 
 1   level1      3573 non-null   object
 2   name        3573 non-null   object
dtypes: int64(1), object(2)
memory usage: 83.9+ KB


In [208]:
product.dtypes

product_id     int64
level1        object
name          object
dtype: object

## 1. По какой категории самое высокое проникновение в трафик? Чему оно равно?

In [209]:
# Подготовим данные:

df = product.merge(order)
df.head()

,product_id,level1,name,order_id,accepted_at,quantity,regular_price,price,cost_price
0,59272,Бакалея,Соус Ben's Original,1518190670,2022-01-15 20:24:33,1,199.0,156.0,120
1,9096,Бакалея,Киноа Агро-Альянс,1517086721,2022-01-11 19:02:24,1,289.0,227.0,125
2,2506,Бакалея,Рис Агро-Альянс,1516814580,2022-01-10 17:21:26,1,122.0,104.0,62
3,2506,Бакалея,Рис Агро-Альянс,1516508527,2022-01-10 17:54:54,4,122.0,104.0,62
4,2506,Бакалея,Рис Агро-Альянс,1518177881,2022-01-15 17:23:12,1,122.0,122.0,62


In [210]:
category = df.level1.value_counts().index[0]

In [211]:
value = df.level1.value_counts().head(1)[0]

In [212]:
print(f'Самое высокое проникновение в трафик по категории {category}, равное {value}')

Самое высокое проникновение в трафик по категории Молочная продукция, равное 2770


## 2. Какой средний чек был 13.01?

In [213]:
# Отберем данные только за 13.01

df_13 = order.query("accepted_at >= '2022-01-13' and accepted_at < '2022-01-14'")
df_13.head()

,order_id,accepted_at,product_id,quantity,regular_price,price,cost_price
10,1517514900,2022-01-13 16:48:19,17122,1,169.0,169.0,74
11,1517430051,2022-01-13 16:46:53,17122,1,169.0,169.0,74
12,1517578174,2022-01-13 18:12:30,17122,1,169.0,169.0,74
17,1517466327,2022-01-13 15:11:12,22199,1,219.0,219.0,130
18,1517429157,2022-01-13 19:15:59,22199,1,219.0,219.0,130


In [214]:
# Создадим колонку с полной стоимостью заказа:

df_13['order_cost'] = df_13.quantity.mul(df_13.price)
df_13.head()

,order_id,accepted_at,product_id,quantity,regular_price,price,cost_price,order_cost
10,1517514900,2022-01-13 16:48:19,17122,1,169.0,169.0,74,169.0
11,1517430051,2022-01-13 16:46:53,17122,1,169.0,169.0,74,169.0
12,1517578174,2022-01-13 18:12:30,17122,1,169.0,169.0,74,169.0
17,1517466327,2022-01-13 15:11:12,22199,1,219.0,219.0,130,219.0
18,1517429157,2022-01-13 19:15:59,22199,1,219.0,219.0,130,219.0


In [215]:
t = df_13[['order_id', 'order_cost']].groupby("order_id").agg(sum).reset_index()

In [216]:
mean_check = t['order_cost'].mean().round(2)

In [217]:
print(f'Средний чек за 13.01.2022 равен {mean_check} руб.')

Средний чек за 13.01.2022 равен 915.64 руб.


## 3. Какая доля промо по категории Сыры?

In [218]:
df.head()

,product_id,level1,name,order_id,accepted_at,quantity,regular_price,price,cost_price
0,59272,Бакалея,Соус Ben's Original,1518190670,2022-01-15 20:24:33,1,199.0,156.0,120
1,9096,Бакалея,Киноа Агро-Альянс,1517086721,2022-01-11 19:02:24,1,289.0,227.0,125
2,2506,Бакалея,Рис Агро-Альянс,1516814580,2022-01-10 17:21:26,1,122.0,104.0,62
3,2506,Бакалея,Рис Агро-Альянс,1516508527,2022-01-10 17:54:54,4,122.0,104.0,62
4,2506,Бакалея,Рис Агро-Альянс,1518177881,2022-01-15 17:23:12,1,122.0,122.0,62


In [219]:
cheese = df.loc[df['level1'] == 'Сыры']

In [220]:
# Оставим только необходимые для расчета столбцы:

cheese = cheese[['product_id', 'regular_price', 'price']]

In [221]:
# Найдем количество промо товаров:

cnt = cheese[cheese['regular_price'] > cheese['price']].shape[0]

In [222]:
print(f'Доля промо в категории "Сыры" равна {round(cnt / cheese.shape[0], 3)}%')

Доля промо в категории "Сыры" равна 0.349%


## 4. Чему равно потребление по категории Птица?

In [223]:
df.head()

,product_id,level1,name,order_id,accepted_at,quantity,regular_price,price,cost_price
0,59272,Бакалея,Соус Ben's Original,1518190670,2022-01-15 20:24:33,1,199.0,156.0,120
1,9096,Бакалея,Киноа Агро-Альянс,1517086721,2022-01-11 19:02:24,1,289.0,227.0,125
2,2506,Бакалея,Рис Агро-Альянс,1516814580,2022-01-10 17:21:26,1,122.0,104.0,62
3,2506,Бакалея,Рис Агро-Альянс,1516508527,2022-01-10 17:54:54,4,122.0,104.0,62
4,2506,Бакалея,Рис Агро-Альянс,1518177881,2022-01-15 17:23:12,1,122.0,122.0,62


In [224]:
birdy = df.loc[df['level1'] == 'Птица', 'quantity'].sum()

In [225]:
print(f'Потребление по категории "Птица" равнa {birdy}')

Потребление по категории "Птица" равнa 541


## 5. Какая маржа в руб и % по категории Молочная продукция?

In [226]:
df.head()

,product_id,level1,name,order_id,accepted_at,quantity,regular_price,price,cost_price
0,59272,Бакалея,Соус Ben's Original,1518190670,2022-01-15 20:24:33,1,199.0,156.0,120
1,9096,Бакалея,Киноа Агро-Альянс,1517086721,2022-01-11 19:02:24,1,289.0,227.0,125
2,2506,Бакалея,Рис Агро-Альянс,1516814580,2022-01-10 17:21:26,1,122.0,104.0,62
3,2506,Бакалея,Рис Агро-Альянс,1516508527,2022-01-10 17:54:54,4,122.0,104.0,62
4,2506,Бакалея,Рис Агро-Альянс,1518177881,2022-01-15 17:23:12,1,122.0,122.0,62


In [227]:
milky = df.loc[df['level1'] == 'Молочная продукция']
milky.head()

,product_id,level1,name,order_id,accepted_at,quantity,regular_price,price,cost_price
10186,105797,Молочная продукция,Творог Самокат,1518152803,2022-01-15 17:21:58,1,125.0,125.0,82
10187,105797,Молочная продукция,Творог Самокат,1517320530,2022-01-12 13:42:06,1,125.0,88.0,82
10188,3414,Молочная продукция,Напиток кисломолочный Actimel,1517443550,2022-01-13 08:08:04,6,27.0,27.0,22
10189,3414,Молочная продукция,Напиток кисломолочный Actimel,1517623373,2022-01-13 16:17:00,3,27.0,27.0,22
10190,3414,Молочная продукция,Напиток кисломолочный Actimel,1516813440,2022-01-10 16:32:30,2,27.0,27.0,22


In [228]:
milky['sum_sale_price'] = milky['price'] * milky['quantity'] # Стоимость, по которой был продан товары
milky['sum_cost_price'] = milky['cost_price'] * milky['quantity'] # Себестоимость товаров

In [229]:
margin = milky['sum_sale_price'].sum() - milky['sum_cost_price'].sum()

In [230]:
pct = margin / milky['sum_sale_price'].sum()

In [231]:
print(f'Маржа в руб равняется {int(margin)}. Маржа в процентах равняется {pct.round(2)}%')

Маржа в руб равняется 106244. Маржа в процентах равняется 0.34%


# Ответим на те же вопросы, но используя SQL и библиотеку sqlite3

Создадим connection к данным

In [232]:
con = sqlite3.connect('CAMOKAT.db')

In [233]:
order = pd.read_csv('/content/drive/MyDrive/Тестовые/Тестовое задание (Ведущий аналитик)_.xlsx - t_order.csv')
products = pd.read_csv('/content/drive/MyDrive/Тестовые/Тестовое задание (Ведущий аналитик)_.xlsx - t_products.csv')

In [234]:
products.head()

,product_id,level1,name
0,59272,Бакалея,Соус Ben's Original
1,9096,Бакалея,Киноа Агро-Альянс
2,2506,Бакалея,Рис Агро-Альянс
3,15017,Бакалея,Укроп Kotanyi
4,823,Бакалея,Макароны Шебекинские


In [235]:
order.to_sql('t_order', con, index=False, if_exists='replace')
products.to_sql('t_products', con, index=False, if_exists='replace')

3573

In [236]:
# Функция для быстрого вывода:

def select(sql):
  return pd.read_sql(sql, con)

In [237]:
order.head()

,order_id,accepted_at,product_id,quantity,regular_price,price,cost_price
0,1516729899,2022-01-10 16:37:37,10163,1,329,329,254
1,1518335746,2022-01-16 12:22:02,10163,1,329,329,229
2,1518354200,2022-01-15 17:03:41,10163,1,329,329,229
3,1516838503,2022-01-11 12:31:53,10163,1,329,329,254
4,1516739758,2022-01-10 19:33:35,10163,1,329,329,254


In [238]:
products.head(2)

,product_id,level1,name
0,59272,Бакалея,Соус Ben's Original
1,9096,Бакалея,Киноа Агро-Альянс


## 1. По какой категории самое высокое проникновение в трафик? Чему оно равно?

In [239]:
sql = """ SELECT level1 as Category,
                 count(level1) as value
          FROM t_order as o
          LEFT JOIN t_products as p ON o.product_id = p.product_id
          GROUP BY level1
          ORDER BY value desc limit 1 """

In [240]:
select(sql)

,Category,value
0,Молочная продукция,2770


## 2. Какой средний чек был 13.01?

In [241]:
sql = """ WITH to_find_mean as (SELECT *, sum(quantity * price) as summ
                                FROM t_order
                                WHERE accepted_at like '2022-01-13%'
                                GROUP BY order_id)

          SELECT ROUND(AVG(summ), 2) as mean_check
          FROM  to_find_mean """

In [242]:
select(sql)

,mean_check
0,915.64


## 3. Какая доля промо по категории Сыры?

In [243]:
sql = """ WITH cheese as (SELECT o.product_id as product_id,
                                 CAST(o.regular_price as REAL) as regular_price,
                                 CAST(o.price as REAL) as price
                          FROM t_order as o
                          LEFT JOIN t_products as p on o.product_id = p.product_id
                          WHERE level1 = 'Сыры')

          SELECT ROUND(CAST(count(*) as REAL) / (SELECT count(*) FROM cheese), 3) as promo
          FROM cheese
          WHERE regular_price > price """

In [244]:
select(sql)

,promo
0,0.349


## 4. Чему равно потребление по категории Птица?

In [245]:
sql = """ SELECT sum(quantity) as birdy
          FROM t_order as o left join t_products as p ON o.product_id = p.product_id
          WHERE level1 = 'Птица' """

In [246]:
select(sql)

,birdy
0,541


## Какая маржа в руб и % по категории Молочная продукция?

In [247]:
sql = """ WITH milky as (SELECT *
                         FROM t_order as o
                         LEFT JOIN t_products as p ON o.product_id = p.product_id
                         WHERE level1 = 'Молочная продукция')

         SELECT ROUND(CAST(margin_rub as REAL) / cnt, 2) as margin_pct,
                (SELECT sum(price * quantity) - sum(cost_price * quantity)
                 FROM milky) as margin_rub
         FROM (SELECT CAST(sum(price * quantity) - sum(cost_price * quantity) AS REAL) AS margin_rub,
                      sum(price * quantity) AS cnt
               FROM milky) AS subquery """

In [248]:
select(sql)

,margin_pct,margin_rub
0,0.34,106243
